In [19]:
import numpy as np
import pandas as pd
import eikon as ek
ek.set_app_key('b4229eae2c644ec8a0aab0bb0fb83b13a38eb2c7')
import cufflinks as cf
import configparser as cp
cf.set_config_file(offline = True)
from datetime import datetime
from datetime import timedelta
import scipy as sp
import scipy.stats as si
import sympy as sy
import sympy.stats as systats
from IPython.display import display_html
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pandas.plotting import table 
import matplotlib
import seaborn as sns
from IPython.display import HTML
from arch import arch_model
from numpy.linalg import inv
from scipy.optimize import fmin_slsqp

import plotly.express as px
import plotly.graph_objects as go
import chart_studio
from chart_studio.plotly import plot, iplot
import chart_studio.plotly as py
from PIL import Image as im
#import Lido_funcs3 as lf

import time


import IPython.display
from IPython.display import Image
chart_studio.tools.set_credentials_file(username='JGarden79', api_key='eWGoAmjzRp3GIVRTFfSR')

import chart_studio.tools as tls

import plotly.io as pio

import plotly.figure_factory as ff
import DatastreamDSWS as DSWS
ds = DSWS.Datastream(username = 'JGarden1@lidoadvisors.com', password= 'Welcome2')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from plotly.subplots import make_subplots

import os
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from scipy.stats.mstats import winsorize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics

from sklearn.cluster import KMeans

In [2]:
def get_clean_tts():
    '''Thsi function will get and clean data for the models'''
    hold = ek.get_data('SPY', fields = [ek.TR_Field('TR.ETPConstituentRIC',params={'SDate':'2020-10-06'})])[0]
    hold = hold[hold['Constituent RIC'] != 'GOOG.OQ']
    rics = [x for x in hold['Constituent RIC']]
    if 'GOOG.OQ' in rics == True:
        rics.remove('GOOG.OQ')
    fields = [ek.TR_Field('TR.GICSSectorCode'),ek.TR_Field('TR.FwdPE', params={'Period':'NTM'}), ek.TR_Field('TR.PriceToBVPerShare'), ek.TR_Field('TR.EVToOperatingCashFlow')]
    data = ek.get_data(rics, fields = fields)[0] ####Bring in valuation metrics
    data = data[data['GICS Sector Code']!='40'] #### Remove Fin
    data = data[data['GICS Sector Code']!='60'] #### Remove RE
    data.to_pickle('data_master.pkl')
    pure_val = ek.get_data('RPV', fields = [ek.TR_Field('TR.ETPConstituentRIC',params={'SDate':'2020-10-07'})])[0] ####Import HOLDINGS of Pure Value
    test_rics_val = [x for x in pure_val['Constituent RIC']] #### Move holdings to list for later use
    time.sleep(15) ###sleep 
    pure_gr = ek.get_data('RPG', fields = [ek.TR_Field('TR.ETPConstituentRIC',params={'SDate':'2020-10-07'})])[0] ####Import HOLDINGS of Pure Growth
    test_rics_gro = [x for x in pure_gr['Constituent RIC']]#### Move to list for later use

    train = pd.DataFrame() ####Establish empty DF

    ###Create the Actual classifications of "PURE VALUE/GROWTH" using a Binary Classifier
    for i in range(0, len(test_rics_val)):####Iterate over a range equal to the number of rics in Value
        x = data[data['Instrument'] == test_rics_val[i]] ###look for each "PURE VALUE" RIC in the complete dataset
        x['Class'] = 1 #### Add a classification value in a new column 1 for value 0 for growth
        train = pd.concat([train, x]) ###add the observation (row) to the training data frame
    
    for i in range(0, len(test_rics_gro)): ###***SEE ABOVE***
        x = data[data['Instrument'] == test_rics_gro [i]]
        x['Class'] = 0
        train = pd.concat([train, x])
    
    train = train.dropna()####Remove missing data


    train = train.sample(frac=1, replace=True, random_state = 42)#### here we create our training data set by randomly selecting data from the training data frame, every item selected is replaced 
    train.to_pickle('training_master.pkl')
    X = train.drop(['Instrument','Class', 'GICS Sector Code'], axis = 1)###This is a DF of features to be fed into the model
    X['Forward P/E (Daily Time Series Ratio)'] = winsorize(X['Forward P/E (Daily Time Series Ratio)'].to_numpy(), limits=[0.1,0.1])
    X['Price To Book Value Per Share (Daily Time Series Ratio)'] = winsorize(X['Price To Book Value Per Share (Daily Time Series Ratio)'].to_numpy(), limits=[0.1,0.1])
    X['Enterprise Value To Operating Cash Flow (Daily Time Series Ratio)'] = winsorize(X['Enterprise Value To Operating Cash Flow (Daily Time Series Ratio)'].to_numpy(), limits=[0.1,0.1])
    sect = train['GICS Sector Code'].to_list()
    #X['GICS Sector Code'] = sect
    y = train['Class'] ### These are classifications used to train and test the model befor we apply to our out of sample 


    X_train, X_test, y_train,y_test = train_test_split(X, y) #### randomly split test data into training and testing data
    
    
    
    
    return X_train, X_test, y_train,y_test

In [3]:
def get_desc():
    df = pd.read_pickle('data_master.pkl')
    codes = [15, 55, 25, 30, 20, 60, 40, 35, 10, 45, 50]
    sectors = ['Mat', 'Utl', 'Disc', 'Stap', 'Ind', 'RE', 'Fin', 'HC', 'Enr', 'Tech', 'CS']
    df_desc = pd.DataFrame()
    
    for i in range(0, len(codes)):
        x = df[df['GICS Sector Code']==codes[i]]
        x['GICS Sector Code'] = [sectors[i] for j in x['GICS Sector Code']]
        df_desc = pd.concat([df_desc, x])
    df_desc = df_desc[df_desc['GICS Sector Code']!= 'Fin']
    df_desc = df_desc[df_desc['GICS Sector Code']!= 'RE']
    
    
    sectors_ex = ['Mat', 'Utl', 'Disc', 'Stap', 'Ind', 'HC', 'Enr', 'Tech', 'CS']
    colors = ["#0466c8", "#0353a4", "#023e7d", "#002855", "#001845", '#001233', "#33415c", "#5c677d", "#7d8597"]
    fig_1 = go.Figure()
    
    
    for i in range(0,len(sectors_ex)):
        df_x = df_desc[df_desc['GICS Sector Code'] == sectors_ex[i]]
        mult = df_x['Forward P/E (Daily Time Series Ratio)'].to_numpy()
        win_mult = winsorize(mult,  limits=[0.1,0.1])
        y = win_mult
        fig_1.add_trace(go.Box(
            y = y,
            name = sectors_ex[i],
            marker_color = colors[i],
            boxpoints=False,
            boxmean = 'sd'))
        
    fig_1.update_layout(title='Forward P/E Ratios by Sector',
                        paper_bgcolor='white',
                        plot_bgcolor='white',)
    
    
    fig_1.update_yaxes(title_text = 'FWD P/E', showgrid=True, gridwidth=1, gridcolor='#e5e5e5',tickfont = {'color': 'black'},title_font = {'color': 'black'}, range = [-75, 220])
    
    fig_2 = go.Figure()
    
    
    for i in range(0,len(sectors_ex)):
        df_x = df_desc[df_desc['GICS Sector Code'] == sectors_ex[i]]
        mult = df_x['Price To Book Value Per Share (Daily Time Series Ratio)'].to_numpy()
        win_mult = winsorize(mult,  limits=[0.1,0.1])
        y = win_mult
        fig_2.add_trace(go.Box(
            y = y,
            name = sectors_ex[i],
            marker_color = colors[i],
            boxpoints=False,
            boxmean = 'sd'))
        
    fig_2.update_layout(title='Price To Book Value Per Share by Sector',
                        paper_bgcolor='white',
                        plot_bgcolor='white',)
    
    
    fig_2.update_yaxes(title_text = 'P/B', showgrid=True, gridwidth=1, gridcolor='#e5e5e5',tickfont = {'color': 'black'},title_font = {'color': 'black'}, range = [-10, 35])
    
    fig_3 = go.Figure()
    for i in range(0,len(sectors_ex)):
        df_x = df_desc[df_desc['GICS Sector Code'] == sectors_ex[i]]
        mult = df_x['Enterprise Value To Operating Cash Flow (Daily Time Series Ratio)'].to_numpy()
        win_mult = winsorize(mult,  limits=[0.1,0.1])
        y = win_mult
        fig_3.add_trace(go.Box(
            y = y,
            name = sectors_ex[i],
            marker_color = colors[i],
            boxpoints=False,
            boxmean = 'sd'))
        
        
    fig_3.update_layout(title='Enterprise Value To Operating Cash Flow by Sector',
                        paper_bgcolor='white',
                        plot_bgcolor='white',)
    
    
    fig_3.update_yaxes(title_text = 'EV/OCF', showgrid=True, gridwidth=1, gridcolor='#e5e5e5',tickfont = {'color': 'black'},title_font = {'color': 'black'}, range = [-5,105])
    
    
        
    
    pio.write_html(fig_1, file='P_E.html', auto_open=True) 
    pio.write_html(fig_2, file='P_B.html', auto_open=True)
    pio.write_html(fig_3, file='EV_FCF.html', auto_open=True)

    
    return 

In [4]:
X_train, X_test, y_train,y_test = get_clean_tts()

In [5]:
get_desc()

In [6]:
###Set up data for GBDecision Tree Train and Test
def GBDT_Model(X_train, X_test, y_train,y_test ):
    clf = GradientBoostingClassifier(learning_rate=0.01, n_estimators=150, random_state=42, max_depth=2).fit(X_train, y_train)
    return clf


In [7]:
def get_GBDT_acc(mod):
    accuracy_train = mod.score(X_train, y_train)
    accuracy_test = mod.score(X_test, y_test)
    data = pd.read_pickle('data_master.pkl')
    X_eval = data.drop(['Instrument'], axis = 1).dropna()
    sect = X_eval['GICS Sector Code'].to_list()
    X_eval = X_eval.drop(['GICS Sector Code'], axis=1)
    lims = X_eval.quantile(0.1)
    lims_2 = X_eval.quantile(0.9)
    X_eval = X_eval.clip(lims, lims_2, axis = 1)
    #X_eval['GICS Sector Code'] = sect
    
    OS_res = mod.predict(X_eval)
    OS_probs = mod.predict_proba(X_eval)

    clf_results = data.dropna()
    clf_results['Predictions'] = OS_res
    clf_results['Probs'] = [p[1] for p in OS_probs ]    
    SPYV = ek.get_data('SPYV.K', fields = [ek.TR_Field('TR.ETPConstituentRIC',params={'SDate':'2020-10-07'})])[0]
    actuals = [x for x in SPYV['Constituent RIC']]
    j = clf_results['Instrument'].tolist()
    y_real = []

    for i in j:
        if i in actuals:
            y_real.append(1)
        else:
            y_real.append(0)

    clf_results['Actuals'] = y_real
    accu = metrics.accuracy_score(clf_results['Actuals'], clf_results['Predictions'])
    
    accus = {'Training score': accuracy_train, 'Test Score':accuracy_test, 'Evaluation Score':accu}
    
    fig = go.Figure(go.Bar(
            x=list(accus.values()),
            y=list(accus.keys()),
            orientation='h',
            text = list(accus.values()),
            textposition='auto',
            marker = dict(color = ["#ffd28f", "#97a5d5", "#005083"])))

    fig.update_traces(texttemplate='%{text:.2%}', textposition='auto'),
    fig.update_layout(
        paper_bgcolor='white',
        plot_bgcolor= 'white',
        height = 600,
        width = 678,
        
        font = dict(family= 'pt sans narrow', size = 25),
        title= {'text': "Gradient Boosted Decision Tree Accuracy Scores"},
        xaxis=dict(
            zeroline=False,
            showline=False,
            showticklabels=False,
            showgrid=True))

    #py.iplot(fig_6, filename='fig_6')
    pio.write_html(fig, file='GBD_acc.html', auto_open=True)
    
    hits = []
    for i in range(0, len(clf_results['Predictions'])):
        if clf_results['Predictions'].iloc[i] == clf_results['Actuals'].iloc[i]:
            hits.append('green')
        else:
            hits.append('red')
    
    clf_results['hits'] = hits
    
    clf_results['hits'] = clf_results['hits'].replace(1, 'green')
    clf_results['hits'] = clf_results['hits'].replace(0, 'red')
    clf_results.to_pickle('GBD_results_DF.pkl')
    
    
    feat_names = [ 'Forward P/E', 'P/B', 'EV/OCF']
    feat_imp = list(mod.feature_importances_)

    
    fig_2 = go.Figure(go.Bar(
            x=feat_imp,
            y=feat_names,
            orientation='h',
            text = feat_imp,
            textposition='auto',
            marker = dict(color = ["#264653", "#e9c46a", "#e76f51"])))

    fig_2.update_traces(texttemplate='%{text:.2%}', textposition='auto'),
    fig_2.update_layout(
        paper_bgcolor='white',
        plot_bgcolor= 'white',
        height = 600,
        width = 678,
        
        font = dict(family= 'pt sans narrow', size = 25),
        title= {'text': "Gradient Boosted Decision Tree Feature Importance"},
        xaxis=dict(
            zeroline=False,
            showline=False,
            showticklabels=False,
            showgrid=True))

    #py.iplot(fig_6, filename='fig_6')
    pio.write_html(fig_2, file='GBD_feat.html', auto_open=True)
    

    
    

    
    
    return 

    
    

In [8]:
mod = GBDT_Model(X_train, X_test, y_train,y_test )
get_GBDT_acc(mod)

In [17]:
def pred_vis(mod):
    data = pd.read_pickle('data_master.pkl')
    X_eval = data.drop(['Instrument'], axis = 1).dropna()
    sect = X_eval['GICS Sector Code'].to_list()
    X_eval = X_eval.drop(['GICS Sector Code'], axis=1)
    lims = X_eval.quantile(0.1)
    lims_2 = X_eval.quantile(0.9)
    X_eval = X_eval.clip(lims, lims_2, axis = 1)
    OS_res = mod.predict(X_eval)
    OS_probs = mod.predict_proba(X_eval)[:,1]
    
    fig = px.scatter(
        X_eval, x='Enterprise Value To Operating Cash Flow (Daily Time Series Ratio)', y='Price To Book Value Per Share (Daily Time Series Ratio)',
        color=OS_probs, color_continuous_scale='RdBu',
        symbol=OS_res, symbol_map={0: 'square', 1: 'circle'},
        labels={'symbol': 'label', 'color': 'score of <br>first class'}
    )
    fig.add_shape(type="rect",
        xref="x", yref="y",
        x0=7.8, y0=0.87, x1=16, y1=17.5,
        line = dict(color="#e63946", width=3),
        fillcolor="#f1faee",
        opacity=0.4,
    )

    fig.add_shape(type="rect",
        xref="x", yref="y",
        x0=16, y0=0.87, x1=39, y1=17.5,
        line = dict(color="#1d3557", width=3),
        fillcolor="#457b9d",
        opacity=0.4,
    )
    
    fig.update_traces(marker_size=9, marker_line_width=1.5)
    fig.update_layout(legend_orientation='h')
    fig.update_layout(
        paper_bgcolor='white',
        plot_bgcolor= 'white',

        
        font = dict(family= 'pt sans narrow', size = 15),
        title= {'text': "Prediction Plots"},
       title_x = 0.5)

    #py.iplot(fig_6, filename='fig_6')
    pio.write_html(fig, file='GBD_pred.html', auto_open=True)
    
    return 

In [18]:
pred_vis(mod)

In [59]:
def k_means_mod():
    df = pd.read_pickle('GBD_results_DF.pkl')
    df = df[df['GICS Sector Code']!=40] #### Remove Fin
    df = df[df['GICS Sector Code']!=60] #### Remove RE
    X = df[['Forward P/E (Daily Time Series Ratio)', 'Price To Book Value Per Share (Daily Time Series Ratio)', 'Enterprise Value To Operating Cash Flow (Daily Time Series Ratio)']].to_numpy()
    y = df[['Actuals']] - 1
    Norm_X = MinMaxScaler().fit(X).transform(X)
    kmeans = KMeans(n_clusters = 2, random_state = 42)
    mod = kmeans.fit(Norm_X)
    accu = metrics.accuracy_score(mod.labels_, y.to_numpy())
    df['unsup_res'] = mod.labels_
    
    codes = [15, 55, 25, 30, 20, 60, 40, 35, 10, 45, 50]
    sectors = ['Mat', 'Utl', 'Disc', 'Stap', 'Ind', 'RE', 'Fin', 'HC', 'Enr', 'Tech', 'CS']
    dat = list(df['GICS Sector Code'])
    sec_labs = []
    for d in dat:
        sec_labs.append(sectors[codes.index(d)])
    df['Sector Name'] = sec_labs
    
    for s in 
        
    
    
    
    return df

In [60]:
k_means_mod()

,Instrument,GICS Sector Code,Forward P/E (Daily Time Series Ratio),Price To Book Value Per Share (Daily Time Series Ratio),Enterprise Value To Operating Cash Flow (Daily Time Series Ratio),Predictions,Probs,Actuals,hits,unsup_res,Sector Name
0,SHW.N,15.0,27.807177,15.777830,22.977151,0,0.110348,0,green,0,Mat
1,DTE.N,55.0,18.686658,2.086542,12.639971,1,0.594741,1,green,0,Utl
2,LOW.N,25.0,18.570842,27.744298,10.420535,0,0.450691,1,red,0,Disc
3,WBA.OQ,30.0,8.149876,1.671607,9.214962,1,0.829508,1,green,0,Stap
4,MAR.OQ,25.0,55.906997,173.875782,23.248087,0,0.110348,0,green,1,Disc
...,...,...,...,...,...,...,...,...,...,...,...
498,CI.N,35.0,10.589318,1.681504,13.453214,1,0.833384,1,green,0,HC
499,FIS.N,45.0,22.684657,1.858445,28.949142,0,0.356917,1,red,0,Tech
500,FTNT.OQ,45.0,33.195458,26.948354,17.969174,0,0.110348,0,green,0,Tech
501,TXN.OQ,45.0,27.073011,17.140328,24.968620,0,0.110348,1,red,0,Tech


In [22]:
def confusion():
    df = pd.read_pickle()


    
    
    return k[1][1]

In [23]:
confusion()

NameError: name 'k' is not defined

In [32]:
df = pd.read_pickle('GBD_results_DF.pkl')
df = df[df['Actuals']==1]
df = df[df['Predictions']==1]
df

,Instrument,GICS Sector Code,Forward P/E (Daily Time Series Ratio),Price To Book Value Per Share (Daily Time Series Ratio),Enterprise Value To Operating Cash Flow (Daily Time Series Ratio),Predictions,Probs,Actuals,hits
1,DTE.N,55.0,18.686658,2.086542,12.639971,1,0.594741,1,green
3,WBA.OQ,30.0,8.149876,1.671607,9.214962,1,0.829508,1,green
8,REG.OQ,60.0,42.618570,1.352350,22.746201,1,0.759520,1,green
9,MS.N,40.0,11.338159,1.048837,22329.041259,1,0.792889,1,green
13,BRKb.N,40.0,22.171479,1.326289,15.289192,1,0.817735,1,green
...,...,...,...,...,...,...,...,...,...
486,TRV.N,40.0,13.202962,1.228254,6.745243,1,0.829508,1,green
488,CVS.N,35.0,8.922645,1.277287,9.583006,1,0.829508,1,green
496,CF.N,15.0,25.494780,2.438842,10.417039,1,0.602541,1,green
497,ALXN.OQ,35.0,10.003200,2.524999,10.539046,1,0.608576,1,green


In [90]:
X=list(k['Price To Book Value Per Share (Daily Time Series Ratio)'])
Y = list(k['Enterprise Value To Operating Cash Flow (Daily Time Series Ratio)'])
text = list(k['STOCKS'])
color = ['#00274C' if i==1 else '#FFCB05' for i in list(k['results']) ]

fig = go.Figure()
fig.add_trace(go.Scatter(x=X, y=Y, mode = 'markers', text = text, marker = dict(color = color)))
fig.show()

NameError: name 'k' is not defined

In [49]:
accu2

NameError: name 'accu2' is not defined